In [9]:
import pandas as pd
import os
import re

# ==========================================
# 1. KONFIGURASI FILE
# ==========================================
file_master_xlsx = 'cost & bbm 2022 sd 2025.xlsx'
file_bbm_xlsx = 'BBM AAB.xlsx'

# Dictionary untuk menyimpan mapping dari Clean Name -> Original Name
map_master_original = {}
map_bbm_original = {}

def clean_name(name):
    """
    Membersihkan nama unit dengan logika 'Super Clean'.
    """
    if pd.isna(name): return None
    name_str = str(name).strip().upper()
    
    # Filter kolom sampah
    if name_str == "" or "UNNAMED" in name_str or "EQUIP NAME" in name_str or "NO." in name_str or "TOTAL" in name_str:
        return None
    
    # 1. Standardisasi Typo (Samakan FORKLIFT vs FORKLIF)
    name_str = name_str.replace("FORKLIFT", "FORKLIF")
    
    # 2. Hapus semua simbol -> Hanya Huruf & Angka (Super Clean Key)
    clean_key = re.sub(r'[^A-Z0-9]', '', name_str)
    
    return clean_key

# ==========================================
# 2. PROSES DATA MASTER (Sheet2)
# ==========================================
master_clean_set = set()

print(f"Sedang membaca Data Master dari: {file_master_xlsx} ...")
try:
    if os.path.exists(file_master_xlsx):
        # Header ada di baris ke-2 (index 1)
        df_master = pd.read_excel(file_master_xlsx, sheet_name='Sheet2', header=1)
        
        if 'NAMA ALAT BERAT' in df_master.columns:
            for original_name in df_master['NAMA ALAT BERAT']:
                cleaned = clean_name(original_name)
                if cleaned:
                    master_clean_set.add(cleaned)
                    # Simpan nama asli
                    if cleaned not in map_master_original:
                        map_master_original[cleaned] = str(original_name).strip()
        else:
            print("⚠️ Kolom 'NAMA ALAT BERAT' tidak ditemukan di Master (Sheet2).")
    else:
        print(f"❌ File tidak ditemukan: {file_master_xlsx}")

except Exception as e:
    print(f"❌ Error membaca master: {e}")

print(f"✅ Total Unit Unik di Master: {len(master_clean_set)}")

# ==========================================
# 3. PROSES DATA BBM AAB (Semua Sheet Bulan)
# ==========================================
bbm_clean_set = set()

print(f"Sedang membaca Data BBM dari: {file_bbm_xlsx} ...")
try:
    if os.path.exists(file_bbm_xlsx):
        xls = pd.ExcelFile(file_bbm_xlsx)
        valid_months = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 
                        'JUL', 'AGT', 'SEP', 'OKT', 'NOV', 'DES']
        
        for sheet_name in xls.sheet_names:
            if any(m in sheet_name.upper() for m in valid_months):
                try:
                    # Baca header saja
                    df_temp = pd.read_excel(xls, sheet_name=sheet_name, nrows=0)
                    for original_col in df_temp.columns:
                        original_str = str(original_col).strip().upper()
                        
                        # A. EXCLUDE Unit Tertentu di Awal
                        if original_str.startswith(('GENSET', 'KOMPRESSOR', 'MESIN', 'TANGKI', 'SPBU', 'MOBIL')):
                            continue

                        # B. Manual Mapping (Pengecualian Spesifik)
                        target_clean_key = None
                        
                        # Case 1: FL RENTAL 01 -> FL RENTAL 01 TIMIKA
                        if "FL RENTAL 01" in original_str and "TIMIKA" not in original_str:
                            target_clean_key = clean_name("FL RENTAL 01 TIMIKA")
                        
                        # Case 2: TOBATI -> TOP LOADER KALMAR 35T/TOBATI
                        elif "TOBATI" in original_str and "KALMAR 32T" in original_str:
                            target_clean_key = clean_name("TOP LOADER KALMAR 35T/TOBATI")
                            
                        # Case 3: L 8477 UUC (EX.L 9902 UR) -> L 9902 UR / S75
                        elif "L 8477 UUC" in original_str and "L 9902 UR" in original_str:
                            target_clean_key = clean_name("L 9902 UR / S75")
                        
                        else:
                            # Normal processing
                            target_clean_key = clean_name(original_col)

                        if target_clean_key:
                            bbm_clean_set.add(target_clean_key)
                            # Simpan nama asli
                            if target_clean_key not in map_bbm_original:
                                map_bbm_original[target_clean_key] = str(original_col).strip()
                                
                except Exception as e:
                    print(f"   ⚠️ Gagal membaca sheet {sheet_name}: {e}")
            else:
                pass 
                
    else:
        print(f"❌ File tidak ditemukan: {file_bbm_xlsx}")

except Exception as e:
    print(f"❌ Error membaca file BBM: {e}")

print(f"✅ Total Unit Unik di BBM AAB: {len(bbm_clean_set)}")

# ==========================================
# 4. FILTER LANJUTAN (RE-CHECK LOGIC)
# ==========================================
print("\nMelakukan pengecekan lanjutan (Logic EX. dan Logic Kurung)...")

initial_missing_keys = sorted(list(bbm_clean_set - master_clean_set))
final_missing_keys = []

for key in initial_missing_keys:
    original_name = map_bbm_original.get(key, "").upper()
    is_found = False
    
    # ---------------------------------------------------------
    # STRATEGI 1: Cek nama SETELAH kata "EX."
    # Contoh: "UNIT BARU (EX. UNIT LAMA)" -> Cek "UNIT LAMA"
    # ---------------------------------------------------------
    if not is_found and "EX." in original_name:
        try:
            part_after_ex = original_name.split("EX.")[-1].replace(")", "").strip()
            clean_part_after = clean_name(part_after_ex)
            
            # Cek Exact Match di Master Clean Set
            if clean_part_after in master_clean_set:
                is_found = True
            
            # Cek Partial Match (jika nama di master mengandung nama belakang ini)
            # Berguna untuk kasus seperti "L 9902 UR" yang di master mungkin "L 9902 UR / S75"
            # Logic: Apakah clean_part_after ada di dalam salah satu key master?
            elif clean_part_after:
                 for master_key in master_clean_set:
                     if clean_part_after in master_key:
                         is_found = True
                         break
                         
        except:
            pass

    # ---------------------------------------------------------
    # STRATEGI 2: Cek nama SEBELUM tanda " ("
    # Contoh: "L 8060 UR (EX.L 9200 UN)" -> Cek "L 8060 UR"
    # ---------------------------------------------------------
    if not is_found and " (" in original_name:
        try:
            part_before_bracket = original_name.split(" (")[0].strip()
            clean_part_before = clean_name(part_before_bracket)
            
            if clean_part_before in master_clean_set:
                is_found = True
        except:
            pass

    # Jika setelah semua cek masih belum ketemu, baru anggap missing
    if not is_found:
        final_missing_keys.append(key)

# ==========================================
# 5. EXPORT KE EXCEL
# ==========================================

missing_in_master_names = [map_bbm_original.get(key, key) for key in final_missing_keys]

# Untuk BBM missing di Master tidak perlu cek logic terbalik, pakai logic standar
missing_in_bbm_keys = sorted(list(master_clean_set - bbm_clean_set))
missing_in_bbm_names = [map_master_original.get(key, key) for key in missing_in_bbm_keys]

output_filename = 'Hasil_Cek_Unit_Revisi2.xlsx'
print(f"Sedang membuat file output: {output_filename}...")

try:
    with pd.ExcelWriter(output_filename) as writer:
        pd.DataFrame({
            'Nama Unit (Original BBM)': missing_in_master_names,
            'ID Pembanding (Clean)': final_missing_keys
        }).to_excel(writer, sheet_name='Missing in Master', index=False)
        
        pd.DataFrame({
            'Nama Unit (Original Master)': missing_in_bbm_names,
            'ID Pembanding (Clean)': missing_in_bbm_keys
        }).to_excel(writer, sheet_name='Missing in BBM AAB', index=False)

    print(f"\n🎉 Selesai! Hasil disimpan di: {output_filename}")
    print(f"- Jumlah Unit di BBM tapi tidak di Master: {len(missing_in_master_names)}")
    print(f"- Jumlah Unit di Master tapi tidak di BBM: {len(missing_in_bbm_names)}")
    
except Exception as e:
    print(f"❌ Gagal menyimpan file excel: {e}")

Sedang membaca Data Master dari: cost & bbm 2022 sd 2025.xlsx ...
✅ Total Unit Unik di Master: 247
Sedang membaca Data BBM dari: BBM AAB.xlsx ...
✅ Total Unit Unik di BBM AAB: 248

Melakukan pengecekan lanjutan (Logic EX. dan Logic Kurung)...
Sedang membuat file output: Hasil_Cek_Unit_Revisi2.xlsx...

🎉 Selesai! Hasil disimpan di: Hasil_Cek_Unit_Revisi2.xlsx
- Jumlah Unit di BBM tapi tidak di Master: 7
- Jumlah Unit di Master tapi tidak di BBM: 48


# CEK 3 FILE

In [10]:
import pandas as pd
import numpy as np
import os
import re
import warnings
import glob
import gc

warnings.filterwarnings('ignore')

# ==============================================================================
# FUNGSI PEMBERSIH NAMA UNIT (GLOBAL)
# ==============================================================================
def clean_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    
    # 1. Standardisasi Typo (Samakan FORKLIFT vs FORKLIF)
    name = name.replace("FORKLIFT", "FORKLIF")
    
    # 2. Hapus semua simbol (/, ., -, spasi) -> Hanya Huruf & Angka
    # Contoh: "FORKLIF MITS/KENANGA" -> "FORKLIFMITSKENANGA"
    # Contoh: "FORKLIFT MITS KENANGA" -> "FORKLIFMITSKENANGA"
    name = re.sub(r'[^A-Z0-9]', '', name)
    
    return name

# ==============================================================================
# BAGIAN 1: LOAD MASTER DATA & GROUPING STRATEGY
# ==============================================================================
print("🚀 [1/3] MEMBACA & MEMBERSIHKAN MASTER DATA ALAT BERAT...")

master_file = 'cost & bbm 2022 sd 2025.xlsx'
# Dictionary Utama: {CLEAN_NAME : {INFO...}}
master_data_map = {} 

try:
    if os.path.exists(master_file):
        df_map = pd.read_excel(master_file, header=1)
        
        df_map.rename(columns={
            'NAMA ALAT BERAT': 'Unit_Original',
            'DES 2025': 'Lokasi',
            'ALAT BERAT': 'Jenis_Alat',
            'CAP': 'Capacity'
        }, inplace=True)
        
        # Filter Kolom Penting
        df_map = df_map[['Unit_Original', 'Lokasi', 'Jenis_Alat', 'Capacity']].copy()
        df_map.dropna(subset=['Unit_Original'], inplace=True)
        
        # --- BUAT CLEAN NAME (ID) ---
        df_map['Unit_ID'] = df_map['Unit_Original'].apply(clean_unit_name)
        
        # Hapus "DUMMY"
        df_map = df_map[~df_map['Unit_Original'].astype(str).str.upper().str.contains('DUMMY', na=False)]
        
        # Pembulatan Kapasitas
        df_map['Capacity'] = pd.to_numeric(df_map['Capacity'], errors='coerce').fillna(0)
        df_map['Capacity'] = np.floor(df_map['Capacity'] + 0.5).astype(int)
        df_map = df_map[df_map['Capacity'] > 0]
        
        print(f"   -> Data Master Terbaca: {len(df_map)} unit valid (Non-Dummy, Cap > 0)")

        # --- STRATEGI GROUPING ---
        def determine_group(row):
            jenis = str(row['Jenis_Alat']).upper().strip()
            cap = row['Capacity']
            
            if jenis in ['FORKLIFT', 'SIDE LOADER']:
                if cap < 5: return f"{jenis} (< 5T)"
                elif cap < 10: return f"{jenis} (5-9T)"
                elif cap < 15: return f"{jenis} (10-14T)"
                else: return f"{jenis} (>= 15T)"
            elif 'CRANE' in jenis:
                if cap < 100: return "CRANE (< 100T)"
                else: return "CRANE (>= 100T)"
            elif jenis in ['REACH STACKER', 'TOP LOADER']:
                if cap > 35: return f"{jenis} (> 35T)"
                else: return f"{jenis} (<= 35T)"
            else:
                return f"{jenis} ({cap}T)"

        df_map['Benchmark_Group'] = df_map.apply(determine_group, axis=1)
        
        # SIMPAN KE DICTIONARY (Agar pencarian O(1) Cepat)
        for _, row in df_map.iterrows():
            master_data_map[row['Unit_ID']] = {
                'Unit_Name': row['Unit_Original'], # Nama Asli Master (Cantik)
                'Benchmark_Group': row['Benchmark_Group'],
                'Lokasi': row['Lokasi'],
                'Jenis_Alat': row['Jenis_Alat'],
                'Capacity': row['Capacity']
            }
            
        print(f"✅ Mapping Selesai. {len(master_data_map)} unit siap dicocokkan.")
        
    else:
        print(f"❌ File '{master_file}' tidak ditemukan.")

except Exception as e:
    print(f"❌ Error membaca file Master: {e}")

🚀 [1/3] MEMBACA & MEMBERSIHKAN MASTER DATA ALAT BERAT...
   -> Data Master Terbaca: 241 unit valid (Non-Dummy, Cap > 0)
✅ Mapping Selesai. 241 unit siap dicocokkan.


In [11]:
# ==========================================
# 2. PROSES DATA MASTER (Sheet2)
# ==========================================
master_clean_set = set()

print(f"Sedang membaca Data Master dari: {file_master_xlsx} ...")
try:
    if os.path.exists(file_master_xlsx):
        # Header ada di baris ke-2 (index 1) sesuai file Anda
        df_master = pd.read_excel(file_master_xlsx, sheet_name='Sheet2', header=1)
        
        if 'NAMA ALAT BERAT' in df_master.columns:
            for original_name in df_master['NAMA ALAT BERAT']:
                cleaned = clean_name(original_name)
                if cleaned:
                    master_clean_set.add(cleaned)
                    # Simpan nama asli agar output Excel rapi
                    if cleaned not in map_master_original:
                        map_master_original[cleaned] = str(original_name).strip()
        else:
            print("⚠️ Kolom 'NAMA ALAT BERAT' tidak ditemukan di Master (Sheet2).")
    else:
        print(f"❌ File tidak ditemukan: {file_master_xlsx}")

except Exception as e:
    print(f"❌ Error membaca master: {e}")

print(f"✅ Total Unit Unik di Master: {len(master_clean_set)}")

Sedang membaca Data Master dari: cost & bbm 2022 sd 2025.xlsx ...
✅ Total Unit Unik di Master: 247


In [12]:
# ==========================================
# 3. PROSES DATA BBM AAB (Semua Sheet Bulan)
# ==========================================
bbm_clean_set = set()

print(f"Sedang membaca Data BBM dari: {file_bbm_xlsx} ...")
try:
    if os.path.exists(file_bbm_xlsx):
        xls = pd.ExcelFile(file_bbm_xlsx)
        valid_months = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 
                        'JUL', 'AGT', 'SEP', 'OKT', 'NOV', 'DES']
        
        for sheet_name in xls.sheet_names:
            if any(m in sheet_name.upper() for m in valid_months):
                try:
                    # Baca header saja (baris pertama = unit names)
                    df_temp = pd.read_excel(xls, sheet_name=sheet_name, nrows=0)
                    for original_col in df_temp.columns:
                        original_str = str(original_col).strip().upper()
                        
                        # --- SYARAT 1: EXCLUDE KATEGORI TERTENTU ---
                        if original_str.startswith(('GENSET', 'KOMPRESSOR', 'MESIN', 'TANGKI', 'SPBU', 'MOBIL')):
                            continue

                        # --- SYARAT 2: MANUAL MAPPING (Unit Bedah Jauh) ---
                        target_clean_key = None
                        
                        # Case A: FL RENTAL 01 -> FL RENTAL 01 TIMIKA
                        if "FL RENTAL 01" in original_str and "TIMIKA" not in original_str:
                            target_clean_key = clean_name("FL RENTAL 01 TIMIKA")
                        
                        # Case B: TOBATI -> TOP LOADER KALMAR 35T/TOBATI
                        elif "TOBATI" in original_str and "KALMAR 32T" in original_str:
                            target_clean_key = clean_name("TOP LOADER KALMAR 35T/TOBATI")
                            
                        # Case C: L 8477 UUC -> L 9902 UR / S75
                        elif "L 8477 UUC" in original_str:
                            target_clean_key = clean_name("L 9902 UR / S75")
                        
                        else:
                            # Normal processing
                            target_clean_key = clean_name(original_col)

                        if target_clean_key:
                            bbm_clean_set.add(target_clean_key)
                            # Simpan nama asli
                            if target_clean_key not in map_bbm_original:
                                map_bbm_original[target_clean_key] = str(original_col).strip()
                                
                except Exception as e:
                    print(f"   ⚠️ Gagal membaca sheet {sheet_name}: {e}")
            else:
                pass 
                
    else:
        print(f"❌ File tidak ditemukan: {file_bbm_xlsx}")

except Exception as e:
    print(f"❌ Error membaca file BBM: {e}")

print(f"✅ Total Unit Unik di BBM AAB: {len(bbm_clean_set)}")

Sedang membaca Data BBM dari: BBM AAB.xlsx ...
✅ Total Unit Unik di BBM AAB: 248


In [13]:
# ==========================================
# 4. FILTER LANJUTAN (LOGIKA EX. & KURUNG)
# ==========================================
print("\nMelakukan pengecekan lanjutan untuk unit missing...")

initial_missing_keys = sorted(list(bbm_clean_set - master_clean_set))
final_missing_keys = []

for key in initial_missing_keys:
    original_name = map_bbm_original.get(key, "").upper()
    is_found = False
    
    # --- LOGIKA 1: CEK SETELAH KATA "EX." ---
    if not is_found and "EX." in original_name:
        try:
            part_after_ex = original_name.split("EX.")[-1].replace(")", "").strip()
            clean_after_ex = clean_name(part_after_ex)
            
            # Cek Exact Match
            if clean_after_ex in master_clean_set:
                is_found = True
            
            # Cek Partial Match (Jika nama master mengandung potongan ini)
            elif clean_after_ex:
                 for master_key in master_clean_set:
                     if clean_after_ex in master_key:
                         is_found = True
                         break
        except:
            pass

    # --- LOGIKA 2: CEK SEBELUM KURUNG " (" ---
    if not is_found and " (" in original_name:
        try:
            part_before_bracket = original_name.split(" (")[0].strip()
            clean_part_before = clean_name(part_before_bracket)
            
            if clean_part_before in master_clean_set:
                is_found = True
        except:
            pass

    # Jika tetap tidak ketemu, masukkan ke daftar Final Missing
    if not is_found:
        final_missing_keys.append(key)

print(f"✅ Pengecekan selesai. Sisa unit missing: {len(final_missing_keys)}")


Melakukan pengecekan lanjutan untuk unit missing...
✅ Pengecekan selesai. Sisa unit missing: 7


In [14]:
# ==========================================
# 5. EXPORT KE EXCEL
# ==========================================

# A. List Missing BBM (Tampilkan nama Asli BBM)
missing_in_master_names = [map_bbm_original.get(key, key) for key in final_missing_keys]

# B. List Missing Master (Tampilkan nama Asli Master)
missing_in_bbm_keys = sorted(list(master_clean_set - bbm_clean_set))
missing_in_bbm_names = [map_master_original.get(key, key) for key in missing_in_bbm_keys]

output_filename = 'Hasil_Cek_Unit.xlsx'
print(f"Sedang membuat file output: {output_filename}...")

try:
    with pd.ExcelWriter(output_filename) as writer:
        # Sheet 1: Ada di BBM, Tapi Master Tidak Punya
        pd.DataFrame({
            'Nama Unit (Original BBM)': missing_in_master_names,
            'ID Pembanding (Clean)': final_missing_keys
        }).to_excel(writer, sheet_name='Missing in Master', index=False)
        
        # Sheet 2: Ada di Master, Tapi BBM Tidak Ada (Inaktif/Rusak)
        pd.DataFrame({
            'Nama Unit (Original Master)': missing_in_bbm_names,
            'ID Pembanding (Clean)': missing_in_bbm_keys
        }).to_excel(writer, sheet_name='Missing in BBM AAB', index=False)

    print(f"\n🎉 Selesai! Hasil disimpan di: {output_filename}")
    print(f"- Jumlah Unit di BBM tapi tidak di Master: {len(missing_in_master_names)}")
    print(f"- Jumlah Unit di Master tapi tidak di BBM: {len(missing_in_bbm_names)}")
    
except Exception as e:
    print(f"❌ Gagal menyimpan file excel: {e}")

Sedang membuat file output: Hasil_Cek_Unit.xlsx...

🎉 Selesai! Hasil disimpan di: Hasil_Cek_Unit.xlsx
- Jumlah Unit di BBM tapi tidak di Master: 7
- Jumlah Unit di Master tapi tidak di BBM: 48


In [15]:
import pandas as pd
import os
import re

# ==========================================
# KONFIGURASI FILE
# ==========================================
file_benchmark = 'Benchmark_Per_Alat_Berat_Data_Baru2.xlsx'
file_bbm = 'BBM AAB.xlsx'
file_master = 'cost & bbm 2022 sd 2025.xlsx'

# ==========================================
# FUNGSI CLEANING (SAMA SEPERTI SEBELUMNYA)
# ==========================================
def clean_name(name):
    if pd.isna(name): return None
    name_str = str(name).strip().upper()
    if name_str == "" or "UNNAMED" in name_str or "EQUIP NAME" in name_str or "NO." in name_str or "TOTAL" in name_str:
        return None
    name_str = name_str.replace("FORKLIFT", "FORKLIF")
    clean_key = re.sub(r'[^A-Z0-9]', '', name_str)
    return clean_key

# ==========================================
# 1. LOAD DATA DARI 3 SUMBER
# ==========================================

# A. DATA BENCHMARK (HASIL ANALISA)
units_benchmark = set()
try:
    if os.path.exists(file_benchmark):
        # Baca Unit Aktif
        df_act = pd.read_excel(file_benchmark, sheet_name='Rapor_Unit_Aktif')
        for u in df_act['Unit_Name']:
            cleaned = clean_name(u)
            if cleaned: units_benchmark.add(cleaned)
        
        # Baca Unit Inaktif
        df_inact = pd.read_excel(file_benchmark, sheet_name='Unit_Inaktif')
        for u in df_inact['Unit_Name']:
            cleaned = clean_name(u)
            if cleaned: units_benchmark.add(cleaned)
            
        print(f"✅ Benchmark: {len(units_benchmark)} unit unik terdeteksi.")
    else:
        print(f"❌ File Benchmark tidak ditemukan: {file_benchmark}")
except Exception as e:
    print(f"❌ Error Benchmark: {e}")

# B. DATA MASTER
units_master = set()
map_master_original = {}
try:
    if os.path.exists(file_master):
        df_master = pd.read_excel(file_master, sheet_name='Sheet2', header=1)
        if 'NAMA ALAT BERAT' in df_master.columns:
            for original_name in df_master['NAMA ALAT BERAT']:
                cleaned = clean_name(original_name)
                if cleaned:
                    units_master.add(cleaned)
                    map_master_original[cleaned] = str(original_name).strip()
        print(f"✅ Master: {len(units_master)} unit unik terdeteksi.")
except Exception as e:
    print(f"❌ Error Master: {e}")

# C. DATA BBM (TRANSAKSI)
units_bbm = set()
map_bbm_original = {}
try:
    if os.path.exists(file_bbm):
        xls = pd.ExcelFile(file_bbm)
        valid_months = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV', 'DES']
        
        for sheet in xls.sheet_names:
            if any(m in sheet.upper() for m in valid_months):
                df_temp = pd.read_excel(xls, sheet_name=sheet, nrows=0)
                for col in df_temp.columns:
                    original_str = str(col).strip().upper()
                    # Filter Junk
                    if original_str.startswith(('GENSET', 'KOMPRESSOR', 'MESIN', 'TANGKI', 'SPBU', 'MOBIL')):
                        continue
                    
                    cleaned = clean_name(col)
                    if cleaned:
                        units_bbm.add(cleaned)
                        map_bbm_original[cleaned] = original_str
                        
        print(f"✅ BBM Transaksi: {len(units_bbm)} unit unik terdeteksi.")
except Exception as e:
    print(f"❌ Error BBM: {e}")

# ==========================================
# 2. LOGIKA PERBANDINGAN (AUDIT)
# ==========================================

# 1. Unit yang ADA di Master TAPI TIDAK ADA di Benchmark
# (Berarti unit ini terdaftar asetnya, tapi tidak masuk laporan analisa - entah karena tidak ada transaksi atau nama beda)
missing_from_analysis = units_master - units_benchmark
missing_analysis_list = []
for u in missing_from_analysis:
    missing_analysis_list.append({
        'Unit_Clean_ID': u,
        'Nama_di_Master': map_master_original.get(u, u),
        'Status_di_BBM': 'Ada Transaksi' if u in units_bbm else 'Tidak Ada Transaksi'
    })

# 2. Unit yang ADA di BBM TAPI TIDAK ADA di Master (Unregistered / Ghost)
# (Ini unit yang beroperasi dan minum solar, tapi tidak ada di database aset Master)
ghost_units = units_bbm - units_master
ghost_units_list = []
for u in ghost_units:
    ghost_units_list.append({
        'Unit_Clean_ID': u,
        'Nama_di_BBM': map_bbm_original.get(u, u)
    })

# 3. Unit yang SUDAH MATCH (Ada di Benchmark)
matched_list = []
for u in units_benchmark:
    matched_list.append({
        'Unit_Clean_ID': u,
        'Nama_di_Master': map_master_original.get(u, "Manual Mapping/Unknown"),
        'Status': 'OK - Masuk Analisa'
    })

# ==========================================
# 3. EXPORT HASIL AUDIT
# ==========================================
output_audit = 'Hasil_Audit_Unit_Lengkap.xlsx'
print(f"\nSedang membuat laporan audit: {output_audit}...")

try:
    with pd.ExcelWriter(output_audit) as writer:
        
        # Sheet 1: Unit Tercecer (Ada Master, Gak Masuk Benchmark)
        if missing_analysis_list:
            pd.DataFrame(missing_analysis_list).to_excel(writer, sheet_name='Master_yg_Belum_Masuk_Analisa', index=False)
        
        # Sheet 2: Unit Hantu (Ada BBM, Gak Ada Master)
        if ghost_units_list:
            pd.DataFrame(ghost_units_list).to_excel(writer, sheet_name='Unit_BBM_Tidak_Dikenal_Master', index=False)
            
        # Sheet 3: Unit Sukses
        if matched_list:
            pd.DataFrame(matched_list).to_excel(writer, sheet_name='Unit_Sukses_Masuk_Analisa', index=False)
            
    print("🎉 Selesai! Silakan cek file output.")
    print(f"   - Master yg Belum Masuk Analisa: {len(missing_analysis_list)} unit")
    print(f"   - Unit BBM Tidak Dikenal Master: {len(ghost_units_list)} unit")
    print(f"   - Unit Sukses Masuk Analisa: {len(matched_list)} unit")

except Exception as e:
    print(f"❌ Gagal save Excel: {e}")

✅ Benchmark: 235 unit unik terdeteksi.
✅ Master: 247 unit unik terdeteksi.
✅ BBM Transaksi: 249 unit unik terdeteksi.

Sedang membuat laporan audit: Hasil_Audit_Unit_Lengkap.xlsx...
🎉 Selesai! Silakan cek file output.
   - Master yg Belum Masuk Analisa: 12 unit
   - Unit BBM Tidak Dikenal Master: 52 unit
   - Unit Sukses Masuk Analisa: 235 unit


In [16]:
import pandas as pd
import os
import re

# ==========================================
# 1. KONFIGURASI FILE
# ==========================================
file_bbm = 'BBM AAB.xlsx'
file_benchmark = 'Benchmark_Per_Alat_Berat_Data_Baru2.xlsx'
file_master = 'cost & bbm 2022 sd 2025.xlsx'

# ==========================================
# 2. FUNGSI CLEANING (STANDAR)
# ==========================================
def clean_name(name):
    if pd.isna(name): return None
    name_str = str(name).strip().upper()
    
    # Filter Sampah
    if name_str == "" or "UNNAMED" in name_str or "EQUIP NAME" in name_str or "NO." in name_str or "TOTAL" in name_str:
        return None
    
    # Standardisasi
    name_str = name_str.replace("FORKLIFT", "FORKLIF")
    # Hapus Simbol
    clean_key = re.sub(r'[^A-Z0-9]', '', name_str)
    
    return clean_key

# ==========================================
# 3. LOAD DATASET (SET BUILDING)
# ==========================================

# --- A. LOAD MASTER (DATABASE ASET) ---
set_master = set()
print(f"1. Membaca Master: {file_master}...")
try:
    if os.path.exists(file_master):
        df = pd.read_excel(file_master, sheet_name='Sheet2', header=1)
        if 'NAMA ALAT BERAT' in df.columns:
            for u in df['NAMA ALAT BERAT']:
                c = clean_name(u)
                if c: set_master.add(c)
except Exception as e:
    print(f"   ❌ Error Master: {e}")

# --- B. LOAD BENCHMARK (HASIL ANALISA) ---
set_benchmark = set()
print(f"2. Membaca Benchmark: {file_benchmark}...")
try:
    if os.path.exists(file_benchmark):
        # Baca Unit Aktif & Inaktif
        for sheet in ['Rapor_Unit_Aktif', 'Unit_Inaktif']:
            try:
                df = pd.read_excel(file_benchmark, sheet_name=sheet)
                for u in df['Unit_Name']:
                    c = clean_name(u)
                    if c: set_benchmark.add(c)
            except: pass
except Exception as e:
    print(f"   ❌ Error Benchmark: {e}")

# --- C. LOAD BBM (TRANSAKSI) DENGAN MAPPING ---
set_bbm = set()
map_bbm_original = {} # Simpan nama asli untuk laporan
print(f"3. Membaca Transaksi BBM: {file_bbm}...")
try:
    if os.path.exists(file_bbm):
        xls = pd.ExcelFile(file_bbm)
        valid_months = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV', 'DES']
        
        for sheet in xls.sheet_names:
            if any(m in sheet.upper() for m in valid_months):
                df_temp = pd.read_excel(xls, sheet_name=sheet, nrows=0)
                for col in df_temp.columns:
                    original_str = str(col).strip().upper()
                    
                    # 1. EXCLUDE
                    if original_str.startswith(('GENSET', 'KOMPRESSOR', 'MESIN', 'TANGKI', 'SPBU', 'MOBIL')):
                        continue
                    
                    # 2. MANUAL MAPPING (Sesuai request sebelumnya)
                    target_clean = None
                    
                    if "FL RENTAL 01" in original_str and "TIMIKA" not in original_str:
                        target_clean = clean_name("FL RENTAL 01 TIMIKA")
                    elif "TOBATI" in original_str and "KALMAR 32T" in original_str:
                        target_clean = clean_name("TOP LOADER KALMAR 35T/TOBATI")
                    elif "L 8477 UUC" in original_str:
                        target_clean = clean_name("L 9902 UR / S75")
                    else:
                        target_clean = clean_name(original_str)
                    
                    if target_clean:
                        set_bbm.add(target_clean)
                        # Simpan nama asli jika belum ada
                        if target_clean not in map_bbm_original:
                            map_bbm_original[target_clean] = original_str
except Exception as e:
    print(f"   ❌ Error BBM: {e}")

# ==========================================
# 4. LOGIKA PERBANDINGAN (GHOST DETECTION)
# ==========================================
print("\nMelakukan Audit Unit Hantu...")

# Unit ada di BBM, TAPI Tidak ada di Master DAN Tidak ada di Benchmark
# (Artinya unit ini lolos filter awal tapi gagal matched ke master, atau master-nya memang tidak punya)
ghost_units_clean = set_bbm - set_master - set_benchmark

# Siapkan Data Output
output_data = []
for clean_id in ghost_units_clean:
    output_data.append({
        'Original_Name_BBM': map_bbm_original.get(clean_id, clean_id),
        'Clean_ID_System': clean_id,
        'Status': 'Ada di BBM, tapi TIDAK ADA di Master & Benchmark'
    })

# ==========================================
# 5. EXPORT HASIL
# ==========================================
output_file = 'Unit_Hantu_Check.xlsx'

if output_data:
    df_out = pd.DataFrame(output_data)
    df_out.sort_values('Original_Name_BBM', inplace=True)
    df_out.to_excel(output_file, index=False)
    
    print(f"\n⚠️ DITEMUKAN {len(output_data)} UNIT MISTERIUS!")
    print(f"File disimpan di: {output_file}")
    print(df_out[['Original_Name_BBM']].head(10))
else:
    print("\n✅ AMAN! Semua unit di BBM sudah terdaftar di Master atau masuk Benchmark.")

1. Membaca Master: cost & bbm 2022 sd 2025.xlsx...
2. Membaca Benchmark: Benchmark_Per_Alat_Berat_Data_Baru2.xlsx...
3. Membaca Transaksi BBM: BBM AAB.xlsx...

Melakukan Audit Unit Hantu...

⚠️ DITEMUKAN 49 UNIT MISTERIUS!
File disimpan di: Unit_Hantu_Check.xlsx
              Original_Name_BBM
36     B 9137 MJ (EX.B 9494 JA)
17                     CONVEY01
26  KOMPESSOR BAN CAB PONTIANAK
0      L 8060 UR (EX.L 9200 UN)
25     L 8073 UL (EX.L 8341 UR)
43     L 8208 UK (EX.L 9436 US)
12     L 8217 UK (EX.L 9069 UT)
48     L 8253 UK (EX.L 9070 UN)
39     L 8271 UK (EX.L 9440 US)
23     L 8280 UK (EX.L 9435 US)
